In [1]:
!pip -q install langchain_community langchain-google-genai faiss-cpu langchain

In [2]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# Basic Retrieval

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_text_splitters import CharacterTextSplitter

In [4]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [5]:
import os
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [6]:
documents = TextLoader("/content/state_of_the_union.txt").load()

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [8]:
texts = text_splitter.split_documents(documents)

In [9]:
embedding = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")

In [10]:
retriever = FAISS.from_documents(texts, embedding).as_retriever()

In [11]:
docs = retriever.invoke("What did the president say about Ketanji Brown Jackson")

In [12]:
pretty_print_docs(docs)

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system.
----------------------------------------------------------------------------------------------------
Docum

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

In [14]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [15]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions using the provided context."),
    ("human", "Context:\n{context}\n\nQuestion:\n{question}")
])

In [16]:
basic_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [17]:
query = "How did the President propose to tackle the issue of climate change?"

In [18]:
print(basic_chain.invoke(query))


To tackle climate change, the President proposed several measures:

*   **Cutting energy costs:** Aiming to save families an average of $500 a year.
*   **Investments and tax credits:** To weatherize homes and businesses for energy efficiency.
*   **Doubling clean energy production:** Specifically in solar, wind, and other clean energy sources.
*   **Lowering electric vehicle prices:** Which could save consumers about $80 a month on gas.
*   **Building EV charging stations:** Establishing a national network of 500,000 electric vehicle charging stations.
*   **Promoting environmental justice:** To withstand the devastating effects of the climate crisis.


# Contextual Compression Retrieval

## LLMChainExtractor

In [19]:
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

In [20]:
compressor = LLMChainExtractor.from_llm(basic_chain)

In [21]:
compression_retriever1=ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [23]:
compressed_docs1 = compression_retriever1.invoke(query)

In [24]:
pretty_print_docs(compressed_docs1)

Document 1:

Second – cut energy costs for families an average of $500 a year by combatting climate change.  

Let’s provide investments and tax credits to weatherize your homes and businesses to be energy efficient and you get a tax credit; double America’s clean energy production in solar, wind, and so much more;  lower the price of electric vehicles, saving you another $80 a month because you’ll never have to pay at the gas pump again.
----------------------------------------------------------------------------------------------------
Document 2:

And we’ll do it all to withstand the devastating effects of the climate crisis and promote environmental justice. 
We’ll build a national network of 500,000 electric vehicle charging stations
----------------------------------------------------------------------------------------------------
Document 3:

NO_OUTPUT.


## LLMChainFilter

In [25]:
from langchain_classic.retrievers.document_compressors import LLMChainFilter

In [26]:
filter = LLMChainFilter.from_llm(llm)

In [27]:
compression_retriever2 = ContextualCompressionRetriever(base_compressor=filter, base_retriever=retriever)

In [29]:
compressed_docs2 = compression_retriever2.invoke(query)

In [30]:
pretty_print_docs(compressed_docs2)

Document 1:

Second – cut energy costs for families an average of $500 a year by combatting climate change.  

Let’s provide investments and tax credits to weatherize your homes and businesses to be energy efficient and you get a tax credit; double America’s clean energy production in solar, wind, and so much more;  lower the price of electric vehicles, saving you another $80 a month because you’ll never have to pay at the gas pump again.
----------------------------------------------------------------------------------------------------
Document 2:

And we’ll do it all to withstand the devastating effects of the climate crisis and promote environmental justice. 

We’ll build a national network of 500,000 electric vehicle charging stations, begin to replace poisonous lead pipes—so every child—and every American—has clean water to drink at home and at school, provide affordable high-speed internet for every American—urban, suburban, rural, and tribal communities. 

4,000 projects have a

## EmbeddingsFilter

In [31]:
from langchain_classic.retrievers.document_compressors import EmbeddingsFilter

In [32]:
embedding_filter = EmbeddingsFilter(embeddings=embedding)

In [33]:
compression_retriever3 = ContextualCompressionRetriever(base_compressor=embedding_filter, base_retriever=retriever)

In [34]:
compressed_docs3 = compression_retriever3.invoke(query)

In [35]:
pretty_print_docs(compressed_docs3)

Document 1:

Second – cut energy costs for families an average of $500 a year by combatting climate change.  

Let’s provide investments and tax credits to weatherize your homes and businesses to be energy efficient and you get a tax credit; double America’s clean energy production in solar, wind, and so much more;  lower the price of electric vehicles, saving you another $80 a month because you’ll never have to pay at the gas pump again.
----------------------------------------------------------------------------------------------------
Document 2:

And we’ll do it all to withstand the devastating effects of the climate crisis and promote environmental justice. 

We’ll build a national network of 500,000 electric vehicle charging stations, begin to replace poisonous lead pipes—so every child—and every American—has clean water to drink at home and at school, provide affordable high-speed internet for every American—urban, suburban, rural, and tribal communities. 

4,000 projects have a

## DocumentCompressorPipeline

In [36]:
from langchain_classic.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

In [37]:
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=".")

In [38]:
redundant_filter = EmbeddingsRedundantFilter(embeddings=embedding)

In [39]:
relevant_filter = EmbeddingsFilter(embeddings=embedding)

In [40]:
pipeline_compressor = DocumentCompressorPipeline(transformers=[splitter, redundant_filter, relevant_filter])

In [41]:
compression_retriever4 = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)

In [42]:
compressed_docs4 = compression_retriever4.invoke(query)

In [43]:
pretty_print_docs(compressed_docs4)

Document 1:

Second – cut energy costs for families an average of $500 a year by combatting climate change
----------------------------------------------------------------------------------------------------
Document 2:

Let’s provide investments and tax credits to weatherize your homes and businesses to be energy efficient and you get a tax credit; double America’s clean energy production in solar, wind, and so much more;  lower the price of electric vehicles, saving you another $80 a month because you’ll never have to pay at the gas pump again
----------------------------------------------------------------------------------------------------
Document 3:

We’ll build a national network of 500,000 electric vehicle charging stations, begin to replace poisonous lead pipes—so every child—and every American—has clean water to drink at home and at school, provide affordable high-speed internet for every American—urban, suburban, rural, and tribal communities
-------------------------------

# NOTE


Full modern RAG example
```
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

compressor = FlashrankRerank(top_n=5)

retrieval_chain = (
    {"context": retriever | compressor,
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

```



🔹 1. LLMChainExtractor

Uses an LLM to extract only relevant spans

Most powerful, most expensive

🔹 2. EmbeddingsFilter

Filters chunks using embedding similarity

Cheap, fast, deterministic

🔹 3. DocumentCompressorPipeline

Chains multiple compressors together

Replacement for “wrapper logic”

🔹 4. FlashrankRerank

Cross-encoder reranker

Great for precision, no LLM cost

**Typical placement**

Retriever
→ EmbeddingsRedundantFilter
→ EmbeddingsFilter
→ FlashrankRerank
→ LLMChainExtractor

